# Modelo del recomedación

## Importación de bases

### Importación de librerías

In [1]:
! pip install numpy cython
! pip install wheel
! pip install scikit-surprise

  Using cached scikit-surprise-1.1.3.tar.gz (771 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp311-cp311-win_amd64.whl size=1296846 sha256=dd3dc97b87bf99ea664b1ecf4df1fa036cdcb14782b96892acd6e16bfc366e80
  Stored in directory: c:\users\dmon2\appdata\local\pip\cache\wheels\f4\2b\26\e2a5eae55d3b7688995e66abe7f40473aac6c95ddd8ee174a8
Successfully built scikit-surprise


In [2]:
# Importación de librerías
import pandas as pd
import numpy as np

from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise import dump

### Importación de bases de datos

In [3]:
# Este notebook fue trabajado desde Google Collabs, debido a dificultades para installar la librería surprise en el computador, por lo que se presentan rutas de archivo diferente para tratar de cargar las bases.

# Desde la pc
# base = pd.read_parquet("../Data/final/total_generos.parquet")

# Reviews
reviews = pd.read_parquet("../Data/final/reviews_final.parquet")
# Games
games = pd.read_parquet("../Data/final/games_final.parquet")
# items
items = pd.read_parquet("../Data/final/items_final.parquet")

# En Google Colabs
# Reviews
#reviews = pd.read_parquet("/content/reviews_final.parquet")
# Games
#games = pd.read_parquet("/content/games_final.parquet")
# items
#items = pd.read_parquet("/content/items_final.parquet")


## Creación del modelo de recomendación

In [4]:
# Se revisa la base de datos de review para que el dataframe cumpla con las
# estructuras que se pasan a surprise para crear un modelo de recomendación.
r = reviews[["user_id","item_id","sentiment_analysis"]]
r

,user_id,item_id,sentiment_analysis
0,76561197970982479,1250,2
1,76561197970982479,22200,1
4,js41637,227300,2
5,js41637,239030,2
6,evcentric,248820,1
...,...,...,...
57629,76561198312638244,233270,2
57630,76561198312638244,130,2
57631,76561198312638244,70,2
57632,76561198312638244,362890,2


In [5]:
# Se crea el reader del modelo para parsear los datos necesarios para la
# recomendación. En este caso, la escala de rating de recomendación se basa en
# "sentiment_analysis", donde el menor dato es 0 y el mayor es 2.
reader = Reader(rating_scale=[0,2])

# Se instancian los datos que se usarán para el modelo de recomendación.
datos = Dataset.load_from_df(r,reader)

In [6]:
# Se crea el modelo de recomendación.
modelo = SVD()

In [7]:
# Se observa una prueba de los resultados del modelo.
cross_validate(modelo,datos,measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.6304  0.6467  0.6368  0.6374  0.6412  0.6385  0.0054  
MAE (testset)     0.5516  0.5597  0.5545  0.5538  0.5553  0.5550  0.0027  
Fit time          1.52    2.00    1.52    1.28    1.34    1.53    0.25    
Test time         0.15    0.19    0.17    0.21    0.19    0.18    0.02    


{'test_rmse': array([0.63042628, 0.64673537, 0.63681204, 0.63740409, 0.64118978]),
 'test_mae': array([0.55158558, 0.55973372, 0.55452148, 0.55383649, 0.55529405]),
 'fit_time': (1.5180845260620117,
  1.9985637664794922,
  1.5179009437561035,
  1.2780039310455322,
  1.3409936428070068),
 'test_time': (0.14581942558288574,
  0.18935847282409668,
  0.1746509075164795,
  0.20516395568847656,
  0.18511128425598145)}

In [8]:
# Se entrena el modelo de recomendación.
entrenamiento = datos.build_full_trainset()
modelo.fit(entrenamiento)

In [9]:
# Se guarda el modelo en un archivo para poder usarlo posteriormente
dump.dump("../Data/modelo/modelo_entrenado.pkl",algo=modelo)

In [10]:
# Se revisa que el modelo cargue adecuadamente.
modelo = dump.load("../Data/modelo/modelo_entrenado.pkl")[1]

In [30]:
# Se crea una primera prueba para obtener las recomendaciones.
def recomendacion_usuario(user_id:str):
    try:
        model = modelo
        # Se obtienen todos los ítems que el usuario no ha calificado.
        items_unrated = [item for item in entrenamiento.all_items() if item not in entrenamiento.ur[user_id]]

        # Se hacen predicciones para los ítems no calificados por el usuario.
        predictions = [model.predict(user_id, item) for item in items_unrated]

        # Se ordenan las predicciones en orden descendente de calificación
        predictions.sort(key=lambda x: x.est, reverse=True)

        # Se obtienen las mejores 5 recomendaciones
        top_n = predictions[:5]

        # Se obtienen los ID de los juegos para cada una de las recomendaciones.
        ids_items = [prediccion.iid for prediccion in top_n]

        # Se instancia la respuesta, devolviendo el ID y el nombre del juego para cada una de las recomendaciones.
        resultado = {
        "item 1":{"id_item":ids_items[0],"game name":games.loc[games["id"] == ids_items[0],"app_name"].iloc[0]},
        "item 2":{"id_item":ids_items[1],"game name":games.loc[games["id"] == ids_items[1],"app_name"].iloc[0]},
        "item 3":{"id_item":ids_items[2],"game name":games.loc[games["id"] == ids_items[2],"app_name"].iloc[0]},
        "item 4":{"id_item":ids_items[3],"game name":games.loc[games["id"] == ids_items[3],"app_name"].iloc[0]},
        "item 5":{"id_item":ids_items[4],"game name":games.loc[games["id"] == ids_items[4],"app_name"].iloc[0]},
        }
        return resultado
    # Se crea una excepción para los casos en donde no se encuentre al id del usuario en la base de datos.
    except Exception:
        return {"No se encontraron datos para el usuario especificado"}

In [29]:
# Se prueba la función.
recomendacion_usuario("js41637")

{'item 1': {'id_item': 2200, 'game name': 'Quake III Arena'},
 'item 2': {'id_item': 630, 'game name': 'Alien Swarm'},
 'item 3': {'id_item': 300, 'game name': 'Day of Defeat: Source'},
 'item 4': {'id_item': 2600,
  'game name': 'Vampire: The Masquerade - Bloodlines'},
 'item 5': {'id_item': 10, 'game name': 'Counter-Strike'}}

In [31]:
# Se prueba la función.
recomendacion_usuario("evcentric")

{'item 1': {'id_item': 300, 'game name': 'Day of Defeat: Source'},
 'item 2': {'id_item': 2270, 'game name': 'Wolfenstein 3D'},
 'item 3': {'id_item': 280, 'game name': 'Half-Life: Source'},
 'item 4': {'id_item': 630, 'game name': 'Alien Swarm'},
 'item 5': {'id_item': 440, 'game name': 'Team Fortress 2'}}

In [32]:
# Se prueba la función.
recomendacion_usuario("76561198312638244")

{'item 1': {'id_item': 630, 'game name': 'Alien Swarm'},
 'item 2': {'id_item': 2500, 'game name': 'Shadowgrounds'},
 'item 3': {'id_item': 2300, 'game name': 'DOOM II'},
 'item 4': {'id_item': 2700, 'game name': 'RollerCoaster Tycoon® 3: Platinum'},
 'item 5': {'id_item': 40, 'game name': 'Deathmatch Classic'}}

In [33]:
# Se prueba la función.
recomendacion_usuario("3215")

{'item 1': {'id_item': 620, 'game name': 'Portal 2'},
 'item 2': {'id_item': 300, 'game name': 'Day of Defeat: Source'},
 'item 3': {'id_item': 10, 'game name': 'Counter-Strike'},
 'item 4': {'id_item': 630, 'game name': 'Alien Swarm'},
 'item 5': {'id_item': 440, 'game name': 'Team Fortress 2'}}

In [36]:
# Se prueba la función.
recomendacion_usuario("A")

{'item 1': {'id_item': 620, 'game name': 'Portal 2'},
 'item 2': {'id_item': 300, 'game name': 'Day of Defeat: Source'},
 'item 3': {'id_item': 10, 'game name': 'Counter-Strike'},
 'item 4': {'id_item': 630, 'game name': 'Alien Swarm'},
 'item 5': {'id_item': 440, 'game name': 'Team Fortress 2'}}